In [1]:
from typing import Union
from datetime import datetime
import json
from copy import deepcopy

from pathlib import Path
import pandas as pd
import xmltodict

In [6]:
class MTConfig:
    def read_json(self, path: Path):
        with open(path, "r", encoding="utf-8") as f:
            config = json.load(f)

        self.__init__(**config)

    def __init__(
        self,
        config_path: Path = None,
        excel_path: Path = None,
        mindmap_path: Path = None,
        color_status_mapping: dict = None,
        shape_type_mapping: dict = None,
    ):
        """
        Initialize the configuration with given parameters,
        or load from a json file at path provided in constructor call (default to current directory).
        """
        self.excel_path = excel_path
        self.mindmap_path = mindmap_path
        self.color_status_mapping = color_status_mapping
        self.shape_type_mapping = shape_type_mapping

        if config_path is not None:
            read_json(config_path)

    def to_json(self, path: Path = None, return_str: bool = True):
        output = deepcopy(self.__dict__)
        output = {k: str(v) for k, v in output.items()}
        if path:
            with open(path, "w", encoding="utf-8") as f:
                json.dump(output, f)
        if return_str:
            return json.dumps(output)

    def __str__(self):
        return str(self.__dict__)

    def __repr__(self):
        return str(self)

In [11]:
class MindTaska:
    def __init__(self, config: Union[MTConfig, Path] = None):
        self.config = config if isinstance(config, MTConfig) else MTConfig(config)
        if self.config.excel_path is not None:
            self.tasks = pd.read_excel(self.config.excel_path, sheet_name="tasks")
            self.stats = pd.read_excel(self.config.excel_path, sheet_name="stats")
        else:
            self.tasks = None
            self.stats = None

    def _parse_node(self, tasks: list, node: dict, project: str, parent: dict = None):
        parent = parent or {}
        result = None
        try:
            result = {
                "task": node["@text"],
                "project": project,
                "type": self.config.shape_type_mapping.get(node.get("@shape"))
                or parent.get("type"),
                "status": self.config.color_status_mapping.get(node.get("@bgColor"))
                or parent.get("status"),
                "worker": node.get("eicon", {}).get("@id"),
            }
            tasks.append(result)
        except KeyError as e:
            print(e)
            print(f"In node :\n{node}")

        topic = node.get("topic", [])
        topic = topic if isinstance(topic, list) else [topic]
        for t in topic:
            self._parse_node(tasks, t, project, result)

    def extract_tasks(self) -> pd.DataFrame:
        with open(self.config.maindmap_path, "r", encoding="utf-8") as f:
            xml_string = f.read()
        xml_data = xmltodict.parse(xml_string)
        tasks = []
        main_node = xml_data["map"]["topic"]["topic"]

        for project in main_node:
            self._parse_node(tasks, project, project["@text"])
        return pd.DataFrame(tasks)

    def get_stat(self, tasks: pd.DataFrame):
        stat = {
            "date": datetime.now(),
            "opened tasks": tasks[tasks["status"] != "complete"].shape[0],
            "new tasks": 0,
            "closed tasks": 0
        }

        for w in tasks["worker"].unique():
            if w is not None:
                stat[w] = tasks[
                    (tasks["worker"] == w) & (tasks["status"] != "complete")
                ].shape[0]

        return pd.DataFrame([stat])
        
    def diff_stat(self, new_tasks: pd.DataFrame):
        if self.config.excel_path is None:
            return new_stats
        
        new_tasks_count = 0
        closed_tasks_count = 0
        for i, t in new_tasks.iterrows():
            same_task = self.tasks[(self.tasks["name"] == t["name"]) & (self.tasks["project"] == t["project"])]
            t_new_count = len(same_tasks)
            if t_new_count == 0:
                new_tasks_count += 0
            elif t_new_count and same_task.iloc[0]["status"] != "complite" and t["status"] == "complete":
                closed_tasks_count += 1
            elif t["status"] == "complite" and same_task.iloc[0]["status"] != "complite":
                new_tasks_count += 1
            
        self.stats["new tasks"] = new_tasks_count
        self.stats["closed tasks"] = closed_tasks_count 